In [20]:
import numpy as np
import pandas as pd
import copy
import random
import time
from util.util_load             import read_txt, read_scenario
from util.util_reschedule       import random_events, store_schedule, update_schedule, snapshot
from util.util_display          import plot, pretty_table
from env_action.metaheu         import GeneticAlgorithm, random_population
from env_action.action_space    import action_space

maxtime                = 1000
PopSize                = 100
critical_machines      = {5, 6, 7, 8, 9, 10, 11, 12, 13, 21, 22, 26, 27}
action_name            = ["exact", "GA", "TS", 
                          "LFOH", "LAPH", "LAP_LFO", 
                          "LFOH_TS", "LAPH_TS", "LFOH_GA", "LAPH_GA",
                          "CDR1", "CDR2", "CDR3", "CDR4", "CDR5", "CDR6",
                          "RCRS"]

J, I, K, p_ijk, h_ijk,   \
d_j, n_j, MC_ji, n_MC_ji,\
OperationPool            = read_txt("DATA/SMALL/Case45_480.txt")
# JA_event, MB_event       = read_scenario("DATA/SMALL/Scenario_1C.txt", K, critical_machines)

variables = ['J', 'p_ijk', 'h_ijk', 'n_j', 'MC_ji', 'n_MC_ji']
for var in variables:
    globals()['org_' + var] = copy.deepcopy(globals()[var])

S_k                    = np.zeros((K))
S_j                    = np.zeros((J))
n_ops_left_j           = copy.deepcopy(n_j)

t                      = 0
JSet                   = list(range(J))
OJSet                  = [[] for _ in range(J)]
for j in JSet:
    OJSet[j]           = [i for i in range(int(n_j[j]))]
Oij_on_machine         = [[] for _ in range(K)]

events                 = {}
MB_record              = {}

# Preschedule
print("preschedule")
StartTime                     = time.time()
population, chromosome_len    = random_population(OperationPool, PopSize)
GBest, X_ijk, S_ij, C_ij, C_j = GeneticAlgorithm(S_k, S_j, JSet, OJSet, J, I, K, 
                                                p_ijk, h_ijk, d_j, n_j, n_ops_left_j, 
                                                MC_ji, n_MC_ji, OperationPool,
                                                PopSize, population, chromosome_len,
                                                StartTime, maxtime=1000)
print("time elapsed", time.time() - StartTime)
fig1 = plot(J, K, n_j, X_ijk, S_ij, C_ij, MB_record, t)
display(fig1)

preschedule
time elapsed 195.43482899665833


In [3]:
A = np.random.randint(0, 10, size = 15)
mask = A > 4
print(A)
print(A[mask])
B = A[mask]
print(B)

[1 7 0 7 5 5 0 7 8 8 6 3 7 9 8]
[7 7 5 5 7 8 8 6 7 9 8]
[7 7 5 5 7 8 8 6 7 9 8]


In [ ]:
# Running schedule
completion = False
# Random events
print("random event")
JA_event, MB_event, t, triggered_event, \
affected_Oij, re, MB_record                           = random_events(t, J, K, X_ijk, S_ij, C_ij, C_j, JA_event, MB_event, MB_record)


while completion == False and triggered_event is not None:
    if t < np.max(C_ij):
        print('----------------------------------------')
        print(t, triggered_event)
        print("snapshot")
        # Snapshot
        S_k, S_j, J, I, JSet, OJSet, DSet, ODSet, OperationPool, \
        n_ops_left_j, MC_ji, n_MC_ji, d_j, n_j, p_ijk, h_ijk,    \
        org_p_ijk, org_h_ijk, org_n_j, org_MC_ji, org_n_MC_ji,   \
        Oij_on_machine, X_ijk, S_ij, C_ij, C_j,                  \
        JA_boolean, JA_long_boolean, JA_urgent_boolean,          \
        MB_boolean, MB_critical_boolean, sum_re,                 \
        CT_k, T_cur, Tard_job, NewJobList            = snapshot(t, triggered_event, MC_ji, n_MC_ji,                 \
                                                                d_j, n_j, p_ijk, h_ijk, J, I, K, X_ijk, S_ij, C_ij, \
                                                                OperationPool, re, S_k,                             \
                                                                org_J, org_p_ijk, org_h_ijk, org_n_j,               \
                                                                org_MC_ji, org_n_MC_ji, C_j                         )
        # Store previous schedule
        print("store schedule")
        X_previous, S_previous, C_previous           = store_schedule(X_ijk, S_ij, C_ij)       
        action_method 						         = action_space(J, I, K, p_ijk, h_ijk, d_j, n_j, 
                                                                    MC_ji, n_MC_ji, n_ops_left_j, OperationPool, S_k, S_j, 
                                                                    JSet, OJSet, Oij_on_machine, affected_Oij, 
                                                                    t, X_ijk, S_ij, C_ij, C_j, CT_k, T_cur, Tard_job,
                                                                    NewJobList, PopSize, maxtime)
        action_id                                    = 0
        print(action_name[action_id])

        reschedule							         = action_method[action_id]
        GBest, X_ijk, S_ij, C_ij, C_j                = reschedule()
        print(GBest)
        print("update_schedule")
        X_ijk, S_ij, C_ij                            = update_schedule(DSet, ODSet, X_ijk, S_ij, C_ij, X_previous, S_previous, C_previous)
        fig2 = plot(J, K, n_j, X_ijk, S_ij, C_ij, MB_record, t)
        display(fig2)
        
        print("random_events")
        JA_event, MB_event, t, triggered_event,\
        affected_Oij, re, MB_record                   = random_events(t, J, K, X_ijk, S_ij, C_ij, C_j, JA_event, MB_event, MB_record)
    else: completion = True


In [ ]:
for k in range(K):
    print(k+1, S_k[k])

In [ ]:
# table = pretty_table(J, I, n_j, X_ijk, S_ij, C_ij)
# table = table[table['Job'] == 13]
# display(table)